In [23]:
import googlemaps
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import re

In [27]:
import sys
sys.path.append('/Users/biryani/Documents/QMUL/Sem C + Thesis/credentials')
import creds

In [7]:
tfl_df = pd.read_csv('../data/station-with-no-coordinates.csv')
tfl_df.head()

,Line,Station Name
0,Bakerloo,Baker Street
1,Bakerloo,Charing Cross
2,Bakerloo,Edgware Road
3,Bakerloo,Elephant & Castle
4,Bakerloo,Embankment


In [28]:
tfl_df_22 = pd.read_csv('../data/2022-tfl-stations.csv')

### In order to fetch the coordinates for each station we are using Google Maps API

In [26]:
gmaps = googlemaps.Client(key=creds.API_KEY)

In [40]:
def get_coordinates(station, line):
    try:
        # First, try with station name and line
        result = gmaps.geocode(f"{station} {line} Line Station, London")
        if result:
            location = result[0]['geometry']['location']
            return location['lat'], location['lng']
        
        # # If not found, try with just station name
        # result = gmaps.geocode(f"{station} Station, London")
        # if result:
        #     location = result[0]['geometry']['location']
        #     return location['lat'], location['lng']
        
        result = gmaps.geocode(f"{station}  Tram Station, London")
        if result:
            location = result[0]['geometry']['location']
            return location['lat'], location['lng']
        
        # If still not found, return None
        return None, None
    except Exception as e:
        print(f"Error fetching coordinates for {station}: {str(e)}")
        return None, None
    
def add_coordinates_to_df(df):
    # Add new columns for latitude and longitude
    df['Latitude'] = None
    df['Longitude'] = None
    
    for index, row in df.iterrows():
        station = row['Station Name']
        line = row['Line']
        
        # Check if we already have coordinates for this station
        if pd.isnull(df.loc[index, 'Latitude']):
            lat, lng = get_coordinates(station, line)
            
            # Update all rows with this station name
            df.loc[df['Station Name'] == station, 'Latitude'] = lat
            df.loc[df['Station Name'] == station, 'Longitude'] = lng
            
            print(f"Added coordinates for {station}: {lat}, {lng}")
            
            # Sleep to avoid hitting API rate limits
            time.sleep(0.1)
    
    return df

In [10]:
tfl_df = add_coordinates_to_df(tfl_df)
tfl_df.head(10)

Added coordinates for Baker Street: 51.5231548, -0.156863
Added coordinates for Charing Cross: 51.50813729999999, -0.1247624
Added coordinates for Edgware Road: 51.5202914, -0.1701944
Added coordinates for Elephant & Castle: 51.5048541, -0.1136621
Added coordinates for Embankment: 51.5032466, -0.112293
Added coordinates for Harlesden: 51.5032466, -0.112293
Added coordinates for Harrow & Wealdstone: 51.59172359999999, -0.3343587
Added coordinates for Kensal Green: 51.5032466, -0.112293
Added coordinates for Kenton: 51.5032466, -0.112293
Added coordinates for Kilburn Park: 51.53504119999999, -0.1938988
Added coordinates for Lambeth North: 51.4989177, -0.1121086
Added coordinates for Maida Vale: 51.5032466, -0.112293
Added coordinates for Marylebone: 51.5032466, -0.112293
Added coordinates for North Wembley: 51.5625616, -0.3175595
Added coordinates for Oxford Circus: 51.51521169999999, -0.1418553
Added coordinates for Paddington: 51.5032466, -0.112293
Added coordinates for Piccadilly Circ

,Line,Station Name,Latitude,Longitude
0,Bakerloo,Baker Street,51.523155,-0.156863
1,Bakerloo,Charing Cross,51.508137,-0.124762
2,Bakerloo,Edgware Road,51.520291,-0.170194
3,Bakerloo,Elephant & Castle,51.504854,-0.113662
4,Bakerloo,Embankment,51.503247,-0.112293
5,Bakerloo,Harlesden,51.503247,-0.112293
6,Bakerloo,Harrow & Wealdstone,51.591724,-0.334359
7,Bakerloo,Kensal Green,51.503247,-0.112293
8,Bakerloo,Kenton,51.503247,-0.112293
9,Bakerloo,Kilburn Park,51.535041,-0.193899


In [41]:
tfl_df_22 = add_coordinates_to_df(tfl_df_22)
tfl_df_22.head(10)

Added coordinates for Baker Street: 51.5231548, -0.156863
Added coordinates for Charing Cross: 51.50813729999999, -0.1247624
Added coordinates for Edgware Road: 51.5202914, -0.1701944
Added coordinates for Elephant & Castle: 51.4946011, -0.100329
Added coordinates for Embankment: 51.5072554, -0.1221937
Added coordinates for Harlesden: 51.53710090000001, -0.2427341
Added coordinates for Harrow & Wealdstone: 51.59172359999999, -0.3343587
Added coordinates for Kensal Green: 51.53054, -0.225247
Added coordinates for Kenton: 51.5865039, -0.3068038
Added coordinates for Kilburn Park: 51.53504119999999, -0.1938988
Added coordinates for Lambeth North: 51.4989177, -0.1121086
Added coordinates for Maida Vale: 51.5299092, -0.1860307
Added coordinates for Marylebone: 51.52248729999999, -0.1630545
Added coordinates for North Wembley: 51.5627879, -0.3041946
Added coordinates for Oxford Circus: 51.51521169999999, -0.1418553
Added coordinates for Paddington: 51.5166744, -0.1769328
Added coordinates fo

,Line,Station Name,Latitude,Longitude
0,Bakerloo,Baker Street,51.523155,-0.156863
1,Bakerloo,Charing Cross,51.508137,-0.124762
2,Bakerloo,Edgware Road,51.520291,-0.170194
3,Bakerloo,Elephant & Castle,51.494601,-0.100329
4,Bakerloo,Embankment,51.507255,-0.122194
5,Bakerloo,Harlesden,51.537101,-0.242734
6,Bakerloo,Harrow & Wealdstone,51.591724,-0.334359
7,Bakerloo,Kensal Green,51.53054,-0.225247
8,Bakerloo,Kenton,51.586504,-0.306804
9,Bakerloo,Kilburn Park,51.535041,-0.193899


In [42]:
def check_coordinates(df):
    missing_coords = df[df['Latitude'].isnull() | df['Longitude'].isnull()]
    
    if missing_coords.empty:
        print("All stations have coordinates.")
    else:
        print(f"{len(missing_coords)} stations are missing coordinates:")
        for _, row in missing_coords.iterrows():
            print(f"- {row['Station Name']} ({row['Line']})")
        
    return missing_coords

missing = check_coordinates(tfl_df)
missing = check_coordinates(tfl_df_22)

All stations have coordinates.
1 stations are missing coordinates:
- Borough (Northern)


In [43]:
tfl_df_22.to_csv('../data/tfl_stations_2022_with_coordinates.csv', index=False)

In [13]:
tfl_df.to_csv('../data/tfl_stations_with_coordinates.csv', index=False)

In [35]:
tfl_df_22.tail(10)

,Line,Station Name,Latitude,Longitude
596,Victoria,Pimlico,51.49521,-0.143898
597,Victoria,Seven Sisters,51.582328,-0.074985
598,Victoria,Stockwell,51.472046,-0.123003
599,Victoria,Tottenham Hale,51.589281,-0.06105
600,Victoria,Vauxhall,51.49521,-0.143898
601,Victoria,Victoria,51.49521,-0.143898
602,Victoria,Walthamstow Central,51.582825,-0.019818
603,Victoria,Warren Street,51.524619,-0.138167
604,Waterloo & City,Bank and Monument,51.511002,-0.086659
605,Waterloo & City,Waterloo,51.503247,-0.112293
